In [1]:
import sys

sys.path.append("..")
from smartdada2.reader.reader import FastqReader

## Reader Fastq File with FastqReader class

Here we are going to use FastqReader to handle any sequence file specific funtionaility.

FastqReader is a memory efficient object to handle sequence data from FastqFiles. The contents within FastqFiles are not loaded into memory thanks to python's generator types. 

Generators are known as "lazy iterators" because they do not iterate until being called. Meaning contents are not loaded until placed in a "for" loop. 

Another thing to be aware of is that generators are not indexable. Unlike lists, you cannot do `generator_object[1]` because index slicing only occurs when an iterator is loaded into memory (which is not the case for generators).

Lastly, generators are not saved in memory after being used. A good analogy will be like a ticketing box. Once all the tickets are gone, they are gone. You would have to store the generator into another variable in order for it to be used again. 

### FastqEntry
FastqReader converts all the contents inside the into FastqEntries, which is a fast and memory efficient in iterative processes. 

This means for every loaded element in a generator is a FastqEntry object.

FastqEntry currently has 4 attributes which are `.header, .seq, .scores and .length`

In [2]:
# Loading in fastq Files
# loading 27K seqs
test_data_file = "../data/SRR1591840.fastq"
reader = FastqReader(test_data_file)

memory_used = sys.getsizeof(reader)/1024**2
total_reads = reader.total_reads()

#total_reads = reader.total_reads()
print(f"Total reads: {total_reads}")
print(f"{memory_used} MB used")


Total reads: 46136
4.57763671875e-05 MB used


/Users/erikserrano/School/CSCI_6611/project/DADA2ParameterExploration/notebooks/../smartdada2/reader/reader.py:173: UserWarning: No reads have been counted. Counting reads now...
  warnings.warn("No reads have been counted. Counting reads now...")


Below are going to be some examples of FastqReader functionalities after loading.

In [3]:
reader.n_entries

0

In [4]:

# iterations 
# only iterating two entries
# -- using enumerate to stop iterating all the entries
# showing all attributes of the FastqEntry Object
for i, entry in enumerate(reader.iter_reads()):
    if i == 2:
        break
    print(f"This is the datatype: {entry.__class__.__name__}")
    print(f"This is the header: {entry.header}")
    print(f"This is the sequence: {entry.seq}")
    print(f"This is the scores: {entry.scores}")
    print(f"This is the length: {entry.length}")
    print()

This is the datatype: FastqEntry
This is the header: @SRR1591840.1.1 1 length=152
This is the sequence: ATACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCATGGCAAGCCAGATGTGAAAGCCCGGGGCTCAACCCCGGGACTGCATTTGGAACTGTCAGGCTAGAGTGTCGGAGAGGAAAGCGGAATTCCTAGT
This is the scores: ????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
This is the length: 152

This is the datatype: FastqEntry
This is the header: @SRR1591840.1.2 1 length=152
This is the sequence: ACCTGTTTGCTCCCCACGCTTTCGAGCCTCAACGTCAGTCCTCGTCCAGAAAGCCGCCTTCGCCACTGGTGTTCCTCCTAATATCTACGCATTTCACCGCTACACTAGGAATTCCGCTTACCTCTCCGACACTCAAGCCTGACAGTTTCCAA
This is the scores: ????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
This is the length: 152



In [5]:
# getting all scores translated into numbers per sequence
# -- this will return a pandas data frame
# the size of the dataframe is represented like: (n_seqs, seq_length)
all_scores_df = reader.get_quality_scores()
print(f"size of dataframe {all_scores_df.shape}")
all_scores_df.head()

size of dataframe (46136, 152)


,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,150,151
0,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
1,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
2,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
3,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
4,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30


In [6]:
# if we are not interested of getting all scores per sequence, we can get the average
# the shape represents average quality score per base pair 
avg_score_series = reader.get_average_score()
print(f"Series shape: f{avg_score_series.shape}")

Series shape: f(152,)


In [7]:
reader.total_reads()

46136